In [1]:
# Constants
DATA_PATH   = "../data/processed/1_preprocessed_df.pkl"
EXPORT_PATH = "../data/processed/3_engineered_df.pkl"

## 2-Feature Engineering

## 3-Feature Selection

In [ ]:
- **Customer_id have alot of unique values, it might not be that useful**

In [7]:
# drop the ratio column
df = df.drop(columns='ratio')

## 4-Feature Scaling

# Export Data

In [ ]:
# Export Data
df.to_pickle(EXPORT_PATH)